In [1]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import inspect
import io 
import json
import numpy as np
import pandas as pd
import requests 
from scipy.optimize import curve_fit


In [2]:
urlData = requests.get('https://corona.lmao.ninja/v2/historical/argentina').content

In [3]:
data = urlData.decode('utf-8')
data

'{"country":"Argentina","province":["mainland"],"timeline":{"cases":{"7/4/20":75376,"7/5/20":77815,"7/6/20":80447,"7/7/20":83426,"7/8/20":87030,"7/9/20":90693,"7/10/20":94060,"7/11/20":97509,"7/12/20":100166,"7/13/20":103265,"7/14/20":106910,"7/15/20":111146,"7/16/20":114783,"7/17/20":119301,"7/18/20":122524,"7/19/20":126755,"7/20/20":130774,"7/21/20":136118,"7/22/20":141900,"7/23/20":148027,"7/24/20":153520,"7/25/20":158334,"7/26/20":162526,"7/27/20":167416,"7/28/20":173355,"7/29/20":178996,"7/30/20":185373,"7/31/20":191302,"8/1/20":196543,"8/2/20":201919},"deaths":{"7/4/20":1481,"7/5/20":1507,"7/6/20":1582,"7/7/20":1644,"7/8/20":1694,"7/9/20":1720,"7/10/20":1774,"7/11/20":1810,"7/12/20":1845,"7/13/20":1903,"7/14/20":1968,"7/15/20":2050,"7/16/20":2112,"7/17/20":2178,"7/18/20":2220,"7/19/20":2260,"7/20/20":2373,"7/21/20":2490,"7/22/20":2588,"7/23/20":2702,"7/24/20":2807,"7/25/20":2893,"7/26/20":2939,"7/27/20":3059,"7/28/20":3179,"7/29/20":3288,"7/30/20":3441,"7/31/20":3543,"8/1/20":359

In [4]:
data_load = json.loads(data)
data_load

{'country': 'Argentina',
 'province': ['mainland'],
 'timeline': {'cases': {'7/4/20': 75376,
   '7/5/20': 77815,
   '7/6/20': 80447,
   '7/7/20': 83426,
   '7/8/20': 87030,
   '7/9/20': 90693,
   '7/10/20': 94060,
   '7/11/20': 97509,
   '7/12/20': 100166,
   '7/13/20': 103265,
   '7/14/20': 106910,
   '7/15/20': 111146,
   '7/16/20': 114783,
   '7/17/20': 119301,
   '7/18/20': 122524,
   '7/19/20': 126755,
   '7/20/20': 130774,
   '7/21/20': 136118,
   '7/22/20': 141900,
   '7/23/20': 148027,
   '7/24/20': 153520,
   '7/25/20': 158334,
   '7/26/20': 162526,
   '7/27/20': 167416,
   '7/28/20': 173355,
   '7/29/20': 178996,
   '7/30/20': 185373,
   '7/31/20': 191302,
   '8/1/20': 196543,
   '8/2/20': 201919},
  'deaths': {'7/4/20': 1481,
   '7/5/20': 1507,
   '7/6/20': 1582,
   '7/7/20': 1644,
   '7/8/20': 1694,
   '7/9/20': 1720,
   '7/10/20': 1774,
   '7/11/20': 1810,
   '7/12/20': 1845,
   '7/13/20': 1903,
   '7/14/20': 1968,
   '7/15/20': 2050,
   '7/16/20': 2112,
   '7/17/20': 2178

In [5]:
df_timeline = pd.json_normalize(data_load['timeline']['cases']).T
df_deaths = pd.json_normalize(data_load['timeline']['deaths']).T
df_recovered = pd.json_normalize(data_load['timeline']['recovered']).T

In [6]:
df_timeline.columns = ['confirmed']
df_deaths.columns = ['deaths']
df_recovered.columns = ['recovered']

In [7]:
data_argentina = pd.concat([df_timeline, df_deaths, df_recovered], axis=1, sort=False)
data_argentina

,confirmed,deaths,recovered
7/4/20,75376,1481,25930
7/5/20,77815,1507,27597
7/6/20,80447,1582,28531
7/7/20,83426,1644,30095
7/8/20,87030,1694,36502
7/9/20,90693,1720,38313
7/10/20,94060,1774,38984
7/11/20,97509,1810,41408
7/12/20,100166,1845,42694
7/13/20,103265,1903,44173


In [8]:
data_argentina.index=pd.to_datetime(data_argentina.index.astype(str), format='%m/%d/%y')

In [9]:
data_argentina['num_id'] = np.arange(len(data_argentina) + 1)[1:]

In [10]:
data_argentina

,confirmed,deaths,recovered,num_id
2020-07-04,75376,1481,25930,1
2020-07-05,77815,1507,27597,2
2020-07-06,80447,1582,28531,3
2020-07-07,83426,1644,30095,4
2020-07-08,87030,1694,36502,5
2020-07-09,90693,1720,38313,6
2020-07-10,94060,1774,38984,7
2020-07-11,97509,1810,41408,8
2020-07-12,100166,1845,42694,9
2020-07-13,103265,1903,44173,10


In [11]:
predicted_days = 7
firstPredictionDay = data_argentina.index[-1] + datetime.timedelta(days = 1)

In [12]:
class Models:    
    def cuadratic_model(x, a, b, c):
        return (a*x**2 + b*x +c)
    
    def cubic_model(x, a, b, c, d):
        return (a*x**3 + b*x**2 +c*x + d)
    
    def fourth_pow_model(x, a, b, c, d, e):
        return (a*x**4 + b*x**3 +c*x**2 + d*x + e)
    
    def fiveth_pow_model(x, a, b, c, d, e, f):
        return (a*x**5 + b*x**4 +c*x**3 + d*x**2 + e*x + f)
    
    def sixh_pow_model(x, a, b, c, d, e, f, g):
        return (a*x**6 + b*x**5 +c*x**4 + d*x**3 + e*x**2 + f*x + g)

In [13]:
def formatPrediction(firstPredictionDay, day, prediction):
    formatedDate = str(firstPredictionDay + datetime.timedelta(days = day))[0:10]
    formatedPrediction = str(int(prediction))
    return "|" + formatedDate + " | " + formatedPrediction + "|"

def printPrediction(model, popt, x, y):
    firstPredictionDayId = x.max() + 1
    print("--- Predictions ---")
    for day in range (0, predicted_days):
        predictionId = firstPredictionDayId + day
        predictionValue = model(predictionId, *popt)
        print(formatPrediction(firstPredictionDay, day, predictionValue))
    print("-------------------")

In [14]:
def printFuncionParameters(model, popt):
        print('\n', model.__name__)
        args = inspect.getfullargspec(model).args[1:]
        for arg, value in zip(args, popt):
            print( arg, " = ", value)

In [15]:
def predictData(model, x, y):
    popt, _ = curve_fit(model, x, y)
    printPrediction(model, popt, x, y)
    printFuncionParameters(model, popt)
    print()

In [16]:
def isModel(methodName):
    return not(methodName.startswith('__'))

for methodName in dir(Models):
    if(isModel(methodName)):
        print(methodName)
        predictData(getattr(Models, methodName), data_argentina.num_id, data_argentina.confirmed)

cuadratic_model
--- Predictions ---
|2020-08-03 | 209368|
|2020-08-04 | 215828|
|2020-08-05 | 222417|
|2020-08-06 | 229136|
|2020-08-07 | 235983|
|2020-08-08 | 242959|
|2020-08-09 | 250065|
-------------------

 cuadratic_model
a  =  64.53467169353668
b  =  2394.391806456051
c  =  73124.71630456187

cubic_model
--- Predictions ---
|2020-08-03 | 209159|
|2020-08-04 | 215538|
|2020-08-05 | 222033|
|2020-08-06 | 228644|
|2020-08-07 | 235370|
|2020-08-08 | 242210|
|2020-08-09 | 249163|
-------------------

 cubic_model
a  =  -0.1279558032455792
b  =  70.48461631130893
c  =  2319.3969180417716
d  =  73334.15431506585

fiveth_pow_model
--- Predictions ---
|2020-08-03 | 207284|
|2020-08-04 | 212447|
|2020-08-05 | 217383|
|2020-08-06 | 222039|
|2020-08-07 | 226364|
|2020-08-08 | 230301|
|2020-08-09 | 233790|
-------------------

 fiveth_pow_model
a  =  -0.00011935193282508472
b  =  -0.10749801434675366
c  =  6.853270008951582
d  =  -72.3322432165053
e  =  3349.26858097373
f  =  71496.274659761